In [13]:
import pandas as pd
import numpy as np

In [14]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
import json
import requests

tmdb = TMDb()
tmdb_movie = Movie()
tmdb.api_key = '4606882be56455abf5485c21adf285fb'

Extracting movie data from Wikipedia - 2021

In [15]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"

dfs = pd.read_html(link, header=0)

if len(dfs) >= 6:
    df1 = dfs[2]
    df2 = dfs[3]
    df3 = dfs[4]
    df4 = dfs[5]

    # Combine DataFrames vertically
    df = pd.concat([df1, df2, df3, df4], ignore_index=True)

    # Define the get_genre function
    def get_genre(x):
        genres = []
        result = tmdb_movie.search(x)
        if len(result) > 0:
            movie_id = result[0].id
            response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, tmdb.api_key))
            data_json = response.json()
            if data_json['genres']:
                genre_str = " " 
                for i in range(0, len(data_json['genres'])):
                    genres.append(data_json['genres'][i]['name'])
                return genre_str.join(genres)
        return np.NaN

    # Apply the get_genre function to the 'Title' column
    df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))

    # Print the resulting DataFrame
    print(df)
else:
    print("Not enough DataFrames found on the page.")

             Opening  Opening.1                     Title   
0      J A N U A R Y        7.0                   The 355  \
1      J A N U A R Y        7.0  The Legend of La Llorona   
2      J A N U A R Y        7.0              The Commando   
3      J A N U A R Y        7.0            American Siege   
4      J A N U A R Y       14.0                    Scream   
..               ...        ...                       ...   
324  D E C E M B E R       30.0            Alice, Darling   
325  D E C E M B E R        NaN                       NaN   
326  D E C E M B E R        NaN                       NaN   
327  D E C E M B E R        NaN                       NaN   
328  D E C E M B E R        NaN                       NaN   

                                    Production company   
0    Universal Pictures / Freckle Films / FilmNatio...  \
1                       Saban Films / Ageless Pictures   
2                 Saban Films / Premiere Entertainment   
3                               Ver

In [16]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.,genres
0,J A N U A R Y,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2],NaN,Action Thriller
1,J A N U A R Y,7.0,The Legend of La Llorona,Saban Films / Ageless Pictures,Patricia Harris Seeley (director/screenplay); ...,[3],NaN,Family Animation Fantasy Horror
2,J A N U A R Y,7.0,The Commando,Saban Films / Premiere Entertainment,Asif Akbar (director); Koji Steven Sakai (scre...,[4],NaN,Action Crime Thriller
3,J A N U A R Y,7.0,American Siege,Vertical Entertainment,Edward Drake (director/screenplay); Timothy V....,[5],NaN,Action Adventure Thriller
4,J A N U A R Y,14.0,Scream,Paramount Pictures / Spyglass Media Group / Ra...,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[6],NaN,Horror Mystery Thriller
...,...,...,...,...,...,...,...,...
324,D E C E M B E R,30.0,"Alice, Darling",Lionsgate / Elevation Pictures,Mary Nighy (director); Alanna Francis (screenp...,NaN,[261],Thriller Mystery Drama
325,D E C E M B E R,NaN,NaN,NaN,NaN,NaN,NaN,Drama
326,D E C E M B E R,NaN,NaN,NaN,NaN,NaN,NaN,Drama
327,D E C E M B E R,NaN,NaN,NaN,NaN,NaN,NaN,Drama


In [17]:
df_2022 = df[['Title','Cast and crew','genres']]
df_2022

,Title,Cast and crew,genres
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Family Animation Fantasy Horror
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller
3,American Siege,Edward Drake (director/screenplay); Timothy V....,Action Adventure Thriller
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller
...,...,...,...
324,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...,Thriller Mystery Drama
325,NaN,NaN,Drama
326,NaN,NaN,Drama
327,NaN,NaN,Drama


In [18]:
def get_director(x):
    if isinstance(x, str):
        if " (director)" in x:
            return x.split(" (director)")[0]
        elif " (directors)" in x:
            return x.split(" (directors)")[0]
        elif " (director/screenplay)" in x:
            return x.split(" (director/screenplay)")[0]
    return np.NaN

df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(x))

C:\Users\user\AppData\Local\Temp\ipykernel_8456\859582795.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(x))


In [19]:
def get_actor1(x):
    if isinstance(x, str):
        split_values = x.split("screenplay); ")
        if len(split_values) > 1:
            return split_values[-1].split(", ")[0]
    return np.NaN

df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(x))

def get_actor2(x):
    if isinstance(x, str):
        split_values = x.split("screenplay); ")
        if len(split_values) > 1 and len(split_values[-1].split(", ")) >= 2:
            return split_values[-1].split(", ")[1]
    return np.NaN

df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(x))

def get_actor3(x):
    if isinstance(x, str):
        split_values = x.split("screenplay); ")
        if len(split_values) > 1 and len(split_values[-1].split(", ")) >= 3:
            return split_values[-1].split(", ")[2]
    return np.NaN

df_2022['actor_3_name'] = df_2022['Cast and crew'].map(lambda x: get_actor3(x))


C:\Users\user\AppData\Local\Temp\ipykernel_8456\3000129866.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(x))
C:\Users\user\AppData\Local\Temp\ipykernel_8456\3000129866.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(x))
C:\Users\user\AppData\Local\Temp\ipykernel_8456\3000129866.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [20]:
df_2022

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Family Animation Fantasy Horror,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller,Asif Akbar,Mickey Rourke,Michael Jai White,NaN
3,American Siege,Edward Drake (director/screenplay); Timothy V....,Action Adventure Thriller,Edward Drake,Timothy V. Murphy,Bruce Willis,NaN
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega
...,...,...,...,...,...,...,...
324,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...,Thriller Mystery Drama,Mary Nighy,Anna Kendrick,Kaniehtiio Horn,Charlie Carrick
325,NaN,NaN,Drama,NaN,NaN,NaN,NaN
326,NaN,NaN,Drama,NaN,NaN,NaN,NaN
327,NaN,NaN,Drama,NaN,NaN,NaN,NaN


In [21]:
df_2022 = df_2022.rename(columns={'Title':'movie_title'})

In [22]:
new_df22 = df_2022.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,The 355
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Family Animation Fantasy Horror,The Legend of La Llorona
2,Asif Akbar,Mickey Rourke,Michael Jai White,NaN,Action Crime Thriller,The Commando
3,Edward Drake,Timothy V. Murphy,Bruce Willis,NaN,Action Adventure Thriller,American Siege
4,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,Scream
...,...,...,...,...,...,...
324,Mary Nighy,Anna Kendrick,Kaniehtiio Horn,Charlie Carrick,Thriller Mystery Drama,"Alice, Darling"
325,NaN,NaN,NaN,NaN,Drama,NaN
326,NaN,NaN,NaN,NaN,Drama,NaN
327,NaN,NaN,NaN,NaN,Drama,NaN


In [23]:
# Fill NaN values with 'unknown'

new_df22['actor_2_name'] = new_df22['actor_2_name'].replace(np.nan, 'unknown')
new_df22['actor_3_name'] = new_df22['actor_3_name'].replace(np.nan, 'unknown')

new_df22['movie_title'] = new_df22['movie_title'].str.lower()

new_df22['comb'] = new_df22['actor_1_name'] + ' ' + new_df22['actor_2_name'] + ' '+ new_df22['actor_3_name'] + ' '+ new_df22['director_name'] +' ' + new_df22['genres']

new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,the 355,Jessica Chastain Lupita Nyong'o Penélope Cruz ...
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Family Animation Fantasy Horror,the legend of la llorona,Autumn Reeser Antonio Cupo Danny Trejo Patrici...
2,Asif Akbar,Mickey Rourke,Michael Jai White,unknown,Action Crime Thriller,the commando,Mickey Rourke Michael Jai White unknown Asif A...
3,Edward Drake,Timothy V. Murphy,Bruce Willis,unknown,Action Adventure Thriller,american siege,Timothy V. Murphy Bruce Willis unknown Edward ...
4,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,scream,Melissa Barrera Mason Gooding Jenna Ortega Mat...
...,...,...,...,...,...,...,...
324,Mary Nighy,Anna Kendrick,Kaniehtiio Horn,Charlie Carrick,Thriller Mystery Drama,"alice, darling",Anna Kendrick Kaniehtiio Horn Charlie Carrick ...
325,NaN,NaN,unknown,unknown,Drama,NaN,NaN
326,NaN,NaN,unknown,unknown,Drama,NaN,NaN
327,NaN,NaN,unknown,unknown,Drama,NaN,NaN


In [24]:
new_df22.isna().sum()

director_name    14
actor_1_name     15
actor_2_name      0
actor_3_name      0
genres            1
movie_title       4
comb             24
dtype: int64

In [25]:
#  remove the rows where there are null values
new_df22 = new_df22.dropna(how='any')
new_df22.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [26]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,the 355,Jessica Chastain Lupita Nyong'o Penélope Cruz ...
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Family Animation Fantasy Horror,the legend of la llorona,Autumn Reeser Antonio Cupo Danny Trejo Patrici...
2,Asif Akbar,Mickey Rourke,Michael Jai White,unknown,Action Crime Thriller,the commando,Mickey Rourke Michael Jai White unknown Asif A...
3,Edward Drake,Timothy V. Murphy,Bruce Willis,unknown,Action Adventure Thriller,american siege,Timothy V. Murphy Bruce Willis unknown Edward ...
4,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,scream,Melissa Barrera Mason Gooding Jenna Ortega Mat...
...,...,...,...,...,...,...,...
320,Kasi Lemmons,Naomi Ackie,Stanley Tucci,Ashton Sanders,Music History Drama,whitney houston: i wanna dance with somebody,Naomi Ackie Stanley Tucci Ashton Sanders Kasi ...
321,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Crime,the pale blue eye,Christian Bale Harry Melling Gillian Anderson ...
322,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,Drama,women talking,Rooney Mara Claire Foy Jessie Buckley Sarah Po...
323,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Comedy Drama,a man called otto,Tom Hanks Mariana Treviño Rachel Keller Marc F...


In [28]:
# convert to csv file
new_df22.to_csv('new_data2022.csv',index=False)